<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">


# *Wumpus*
### *Sistemas Inteligentes* (Curso 2024-2025)



<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">Características del juego para segunda convocatoria </h2>

## Objetivos a cubrir en la práctica en segunda convocatoria

En la versión del juego del **Wumpus**, en segunda convocatoria, las características a cubrir son las siguientes:

1. El juego debe permitir necesariamente al usuario poder elegir, **EXCLUSIVAMENTE**, entre los dos siguientes tamaños de tablero: (a) $8\times8$ y (c) $10\times10$.


2. Debe haber dos huecos y un **wumpus**, colocados en posiciones aleatorias, además del oro.


3. <span style="color:red">Cuando NO le corresponde al agente mover, el movimiento consistirá en la elección **ALEATORIA** entre el movimiento de cualquiera de los dos huecos, y el del wumpus. Es decir, en esta versión del juego, el **wumpus** puede moverse.</span>


4. El movimiento de la casilla del **wumpus**, siempre es **aleatorio**.


5. El movimiento, tanto del agente, como de los huecos o el wumpus, es **SIEMPRE** a una casilla contigua a la inicial, de entre las permitidas.


6. <span style="color:red">**Tendréis que cambiar la función de coste**, teniendo en cuenta que el **wumpus** puede moverse y que al caer, el agente **pierde la partida**</span>.



In [1]:
from copy import deepcopy
from typing import Tuple, List
from IPython.display import display, HTML
import math
import random

In [2]:
class Tablerowumpus:
    def __init__(self, size, num_huecos=2):
        if size not in [8, 10]:
            raise ValueError("El tamaño del tablero debe ser 8x8 o 10x10")
            
        self.size = size
        self.previous_content = "B"
        self.flecha_disponible = True
        self.posiciones_visitadas = set()
        self.posiciones_hedor = set()
        self.matrix = self.inicializar_tablero(num_huecos)
        self.TableroIni = deepcopy(self.matrix)
        self.agent_pos = self.posicionAgente()
        self.posiciones_visitadas.add(self.agent_pos)


    def inicializar_tablero(self, num_huecos: int):
        # Exactamente 2 huecos según requisitos
        num_huecos = 2  
        tablero = [["B" for _ in range(self.size)] for _ in range(self.size)]
        posiciones_usadas = set()

        # Colocar agente en la esquina inferior izquierda
        agente_pos = (self.size - 1, 0)
        tablero[agente_pos[0]][agente_pos[1]] = "A"
        posiciones_usadas.add(agente_pos)

        fichas = ['W', 'O'] + ['C'] * num_huecos

        def es_posicion_valida(fila, columna, ficha):
            if (fila, columna) in posiciones_usadas:
                return False
            if ficha == "O" and self.esAdyacente((fila, columna), agente_pos):
                return False 
            if ficha == "C" and self.esAdyacente((fila, columna), agente_pos):
                return False
            if ficha == "W" and self.esAdyacente((fila, columna), agente_pos):
                return False
            return True

        wumpus_pos = None
        for ficha in fichas:
            while True:
                fila = random.randint(0, self.size - 1)
                columna = random.randint(0, self.size - 1)
                if es_posicion_valida(fila, columna, ficha):
                    tablero[fila][columna] = ficha
                    posiciones_usadas.add((fila, columna))
                    if ficha == "W":
                        wumpus_pos = (fila, columna)
                    break

        self.actualizarBrisasYHedores(tablero)
        return tablero

    def esAdyacente(self, pos1, pos2):
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1]) == 1

    def setMatrix(self, matrix):
        self.matrix = deepcopy(matrix)
    
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)

    def posicionAgente(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.matrix[i][j] == 'A':
                    return (i, j)
        return None

    def PosicionOro(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.matrix[i][j] == 'O':
                    return (i, j)
        return None

    def posicioWumpus(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.matrix[i][j] == 'W':
                    return (i, j)
        return None
    
    def endOfGame(self):
        return (self.posicionAgente() is None) or (self.PosicionOro() is None)

    def identificaVecinos(self, posicion):
        if posicion is None:
            return []
        fila, columna = posicion
        vecinos = []
        direccion = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dirf, dirc in direccion:
            nueva_fila = fila + dirf
            nueva_columna = columna + dirc
            if 0 <= nueva_fila < self.size and 0 <= nueva_columna < self.size:
                vecinos.append((nueva_fila, nueva_columna))
        return vecinos

    def possibleMoves(self, caracter, position: Tuple[int, int]) -> List[Tuple[int, int]]:
        movs_posibles = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        movs_validos = []
        row, col = position
        for dx, dy in movs_posibles:
            n_row, n_col = row + dx, col + dy
            if 0 <= n_row < self.size and 0 <= n_col < self.size:
                contenido = self.matrix[n_row][n_col]
                if caracter == 'W':
                    if contenido not in ['O', 'C']:
                        movs_validos.append((n_row, n_col))
                elif caracter == 'C':
                    if contenido not in ['O', 'C', 'W']:
                        movs_validos.append((n_row, n_col))
                else:
                    movs_validos.append((n_row, n_col))

        return movs_validos

    def isGameOver(self, row, col):
        contenido = self.matrix[row][col]
        return contenido in ['W', 'C']

    def turno_elementos(self):
        """Movimiento aleatorio entre Wumpus y huecos"""
        elemento_a_mover = random.choice(['wumpus', 'hueco1', 'hueco2'])
        
        if elemento_a_mover == 'wumpus':
            wumpus_pos = self.posicioWumpus()
            if wumpus_pos:
                movimientos = self.possibleMoves('W', wumpus_pos)
                if movimientos:
                    nueva_pos = random.choice(movimientos)
                    self.matrix[wumpus_pos[0]][wumpus_pos[1]] = 'B'
                    self.matrix[nueva_pos[0]][nueva_pos[1]] = 'W'
        else:
            huecos = [(i, j) for i in range(self.size) 
                     for j in range(self.size) if self.matrix[i][j] == 'C']
            
            if len(huecos) >= 2:
                indice_hueco = 0 if elemento_a_mover == 'hueco1' else 1
                hueco_pos = huecos[indice_hueco]
                movimientos = self.possibleMoves('C', hueco_pos)
                if movimientos:
                    nueva_pos = random.choice(movimientos)
                    self.matrix[hueco_pos[0]][hueco_pos[1]] = 'B'
                    self.matrix[nueva_pos[0]][nueva_pos[1]] = 'C'
        
        self.actualizarBrisasYHedores(self.matrix)

    def moverAgente(self, nueva_pos):
        if nueva_pos and 0 <= nueva_pos[0] < self.size and 0 <= nueva_pos[1] < self.size:
            f, c = self.posicionAgente()
            self.matrix[f][c] = 'B'
            self.matrix[nueva_pos[0]][nueva_pos[1]] = 'A'
            self.posiciones_visitadas.add(nueva_pos)

    def actualizarBrisasYHedores(self, tablero):
        def limpiar_celdas(tipo: str):
            for fila in range(self.size):
                for columna in range(self.size):
                    if tablero[fila][columna] == tipo:
                        tablero[fila][columna] = 'B'

        def aplicar_efecto_alrededor(posicion, efecto):
            fila, columna = posicion
            direcciones = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            for dx, dy in direcciones:
                nueva_fila = fila + dx
                nueva_columna = columna + dy
                if 0 <= nueva_fila < self.size and 0 <= nueva_columna < self.size:
                    if tablero[nueva_fila][nueva_columna] == 'B':
                        tablero[nueva_fila][nueva_columna] = efecto

        limpiar_celdas('S')
        limpiar_celdas('H')

        for fila in range(self.size):
            for columna in range(self.size):
                if tablero[fila][columna] == 'C':
                    aplicar_efecto_alrededor((fila, columna), 'S')
                elif tablero[fila][columna] == 'W':
                    aplicar_efecto_alrededor((fila, columna), 'H')

    def utility(self) -> float:
        agente_pos = self.posicionAgente()
        oro_pos = self.PosicionOro()
        wumpus_pos = self.posicioWumpus()
        
        # Si el agente ha caído en un hueco o ha sido capturado por el Wumpus
        if not agente_pos:
            return float('-inf')  # Penalización máxima por perder
            
        # Si no encontramos el oro (El agente está en la casilla del oro)
        if not oro_pos:
            return float('inf')

        valor_total = 0.0
        
        # 1. Evaluación de la distancia al oro
        dist_oro = abs(agente_pos[0] - oro_pos[0]) + abs(agente_pos[1] - oro_pos[1])
        valor_total += 15000 / (1 + dist_oro)
        
        # 2. Evaluación del riesgo por Wumpus móvil
        if wumpus_pos:
            dist_wumpus = abs(agente_pos[0] - wumpus_pos[0]) + abs(agente_pos[1] - wumpus_pos[1])
            
            if dist_wumpus <= 2:
                valor_total -= 6000 / (1 + dist_wumpus)
            else:
                valor_total -= 3000 / (1 + dist_wumpus)
                
            movimientos_wumpus = self.possibleMoves('W', wumpus_pos)
            for mov in movimientos_wumpus:
                if self.esAdyacente(mov, agente_pos):
                    valor_total -= 4000
        
        # 3. Evaluación de riesgo por huecos
        huecos = [(i, j) for i in range(self.size) 
                for j in range(self.size) if self.matrix[i][j] == 'C']
        
        for hueco in huecos:
            dist_hueco = abs(agente_pos[0] - hueco[0]) + abs(agente_pos[1] - hueco[1])
            
            if dist_hueco <= 2:
                valor_total -= 5000 / (1 + dist_hueco)
            else:
                valor_total -= 2000 / (1 + dist_hueco)
                
            movimientos_hueco = self.possibleMoves('C', hueco)
            for mov in movimientos_hueco:
                if self.esAdyacente(mov, agente_pos):
                    valor_total -= 3000
        
        # 4. Evaluación de percepciones
        vecinos = self.identificaVecinos(agente_pos)
        for vecino in vecinos:
            contenido = self.matrix[vecino[0]][vecino[1]]
            if contenido == 'H':
                valor_total -= 2500
            elif contenido == 'S':
                valor_total -= 2000
                
        # 5. Bonificación por casillas seguras
        casillas_seguras = sum(1 for vecino in vecinos 
                            if self.matrix[vecino[0]][vecino[1]] not in ['W', 'C'])
        valor_total += casillas_seguras * 150
        
        # 6. Penalización por casillas visitadas
        if agente_pos in self.posiciones_visitadas:
            valor_total -= 200
        
        return valor_total

    def mostrar_tablero(self):
        for fila in self.matrix:
            print(" ".join(fila))

# Modo grafico

In [3]:
def getCodigoTablero(caracter):
    if caracter == 'A':  # Agente
        return 1
    elif caracter == 'W':  # Wumpus
        return 2        
    elif caracter == 'H':  # Hedor
        return 3
    elif caracter == 'O':  # Oro
        return 4
    elif caracter == 'S':  # Brisa
        return 5
    elif caracter == 'C':  # Hueco
        return 6
    return 0  # Casilla vacía

def get_content(coord, tablero):
    codigo = getCodigoTablero(tablero[coord[0]][coord[1]])
    contenidos = {
        0: "CasillaVacia",
        1: "CasillaAgente",
        2: "CasillaWumpus",
        3: "CasillaHedor",
        4: "CasillaOro",
        5: "CasillaBrisa",
        6: "CasillaHueco"
    }
    return contenidos[codigo]

element_image = {
    "CasillaVacia": "./ImagenesCasillasWumpus/CasillaVacia.png",
    "CasillaAgente": "./ImagenesCasillasWumpus/CasillaAgente.png",
    "CasillaWumpus": "./ImagenesCasillasWumpus/CasillaWumpus.png",
    "CasillaHedor": "./ImagenesCasillasWumpus/CasillaHedor.png",
    "CasillaOro": "./ImagenesCasillasWumpus/CasillaOro.png",
    "CasillaBrisa": "./ImagenesCasillasWumpus/CasillaBrisa.png",
    "CasillaHueco": "./ImagenesCasillasWumpus/CasillaHueco.png"
}
    
def get_html(tablero):
    """Muestra una representación gráfica del tablero en HTML con imágenes más grandes."""
    height = len(tablero)
    width = len(tablero[0])

    html_string = "<style> img.game {width: 75px !important; height: 45px !important;}</style><table>"
    for i in range(height):
        html_string += "<tr>"
        for j in range(width):
            content = get_content((i, j), tablero)
            image_path = element_image[content] 
            html_string += f'<td><img class="game" src="{image_path}" alt=""></td>'
        html_string += "</tr>"
    html_string += "</table>"

    return html_string

## Función asociada al método *Minimax* (con poda $\alpha-\beta$)

In [4]:
def miniMax(state: Tablerowumpus, currentLevel: int, maxLevel: int, player: int, ultimoMov, alpha: int, beta: int, stop: bool) -> Tuple[List[List], float, bool]:
    # Verificar condiciones de terminación
    if currentLevel == maxLevel or state.endOfGame():
        return (state.matrix, state.utility(), stop)
    
    successorMatrices = []
    
    if player == 1:  # Turno del agente
        # Obtener movimientos posibles para el agente
        agente_pos = state.posicionAgente()
        if agente_pos:
            moves = state.possibleMoves('A', agente_pos)
            for move in moves:
                if move != ultimoMov:
                    clon = deepcopy(state)
                    clon.moverAgente(move)
                    successorMatrices.append(clon)
    else:  # Turno de los elementos (Wumpus y huecos)
        clon = deepcopy(state)
        clon.turno_elementos()
        successorMatrices.append(clon)
    
    # Si no hay movimientos posibles
    if not successorMatrices:
        return (state.matrix, state.utility(), True)
    
    if player == 1:  # MAX
        maxValue = float('-inf')
        bestMatrix = state.matrix
        
        for succ in successorMatrices:
            _, value, _ = miniMax(succ, currentLevel + 1, maxLevel, -1, ultimoMov, alpha, beta, stop)
            if value > maxValue:
                maxValue = value
                bestMatrix = succ.matrix
            alpha = max(alpha, maxValue)
            if beta <= alpha:
                break
        
        return (bestMatrix, maxValue, stop)
    else:  # MIN
        minValue = float('inf')
        bestMatrix = state.matrix
        
        for succ in successorMatrices:
            _, value, _ = miniMax(succ, currentLevel + 1, maxLevel, 1, ultimoMov, alpha, beta, stop)
            if value < minValue:
                minValue = value
                bestMatrix = succ.matrix
            beta = min(beta, minValue)
            if beta <= alpha:
                break
        
        return (bestMatrix, minValue, stop)

---
## Poner en funcionamiento Minimax

Una vez realizada/terminada la función que implementa el algoritmo de **MINIMAX**, podemos *ponerla en funcionamiento*.

In [5]:
def performActionMinMax(state:Tablerowumpus, player:int, ultimoMov):
        
    # ==================================================
    # Valor de profundidad que queráis. Yo recomiendo 2
    
    # depth = ........
    
    depth = 2
    #matrizoptima =
    stop = False 
    currentLevel = 0
    
    (best_move, minValue, stop) = miniMax(state, currentLevel, depth, player, ultimoMov, -math.inf, math.inf, stop)       

    return best_move 

## Realización del movimiento por parte del *agente*

Definimos una función que, cuando se realice, haga el movimiento del *agente*.

In [6]:
def AIAction(state: Tablerowumpus, player: int):
    global AIReadyToMove

    # Obtener la matriz actual del estado del juego
    matriz = state.getMatrix()

    # ======================================================================================================
    #
    # Aquí realizamos la llamada a "performActionMinMax" para obtener el mejor estado
    # Basándonos en el estado actual del tablero y el jugador
    #
    # ======================================================================================================
    # Definir la posición inicial del agente
    ultimoMov = state.posicionAgente()

    # Ejecutar el algoritmo Minimax para determinar el mejor estado
    mejor_estado = performActionMinMax(state, player, ultimoMov)

    # ======================================================================================================
    #
    # Desactivar indicador de movimiento automático
    #
    # ======================================================================================================
    AIReadyToMove = False

    # ======================================================================================================
    #
    # Retornar el estado resultante después de aplicar el movimiento Minimax
    #
    # ======================================================================================================
    return mejor_estado

## El main

In [7]:
def main():
    # Solicitar tamaño del tablero
    while True:
        try:
            size = int(input("Elige el tamaño del tablero (8 o 10): "))
            if size in [8, 10]:
                break
            else:
                print("Por favor, elige un tamaño válido: 8x8 o 10x10.")
        except ValueError:
            print("Entrada inválida. Debes ingresar un número.")

    # Número de huecos fijo en 2 según requisitos
    num_huecos = 2
    print("Se utilizarán 2 huecos según los requisitos del juego.")

    # Inicializar el tablero
    tablero = Tablerowumpus(size, num_huecos)
    print("Tablero inicial:")
    display(HTML(get_html(tablero.matrix)))  # Mostrar el tablero inicial

    # Ejecutar el algoritmo de la IA
    player = 1
    while not tablero.endOfGame():
        if player == 1:
            print("Turno del agente ...")
        else:
            print("Turno de los elementos ...")

        # Usar AIAction para obtener el mejor estado del tablero
        mejor_estado = AIAction(tablero, player)

        # Actualizamos el tablero con el estado retornado por AIAction
        tablero.setMatrix(mejor_estado)

        # Actualizamos la posición del agente
        #tablero.moverAgente(tablero.posicionAgente())
        tablero.actualizarBrisasYHedores(tablero.matrix)

        # Mostrar el tablero después del movimiento
        print("Estado actual del tablero:")
        display(HTML(get_html(tablero.matrix)))

        # Cambiar de jugador
        player = -player

        opcion = input("Presiona Enter para continuar o 'q' para salir: ")
        if opcion.lower() == 'q':
            break

    print("El juego ha terminado.")

In [ ]:
# Ejecutar el juego
if __name__ == "__main__":
    main()